## 📚 Import Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print("✅ Core libraries loaded!")

# Try to import advanced forecasting libraries
try:
    from statsmodels.tsa.arima.model import ARIMA
    from statsmodels.tsa.seasonal import seasonal_decompose
    print("✅ ARIMA available")
    ARIMA_AVAILABLE = True
except ImportError:
    print("⚠️  ARIMA not available (install: pip install statsmodels)")
    ARIMA_AVAILABLE = False

try:
    from prophet import Prophet
    print("✅ Prophet available")
    PROPHET_AVAILABLE = True
except ImportError:
    print("⚠️  Prophet not available (install: pip install prophet)")
    PROPHET_AVAILABLE = False

print("\n📊 Ready for forecasting!")

✅ Core libraries loaded!
✅ ARIMA available
✅ ARIMA available
✅ Prophet available

📊 Ready for forecasting!
✅ Prophet available

📊 Ready for forecasting!


## 📂 Load Economic Data

In [2]:
# Load economic data
data_path = r"D:\project\dragon-fly-data\processdataset\economic_consolidated.csv"
df = pd.read_csv(data_path)

# Convert to numeric
numeric_cols = ['GDPTotalBillion', 'GDPGrowthRate', 'InflationRate', 'GDPPerCapita', 
                'UnemploymentRate', 'FDINetInflowsMillion', 'ExportsPercentGDP']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Focus on post-Doi Moi period (1986+) for more stable patterns
df_model = df[df['Year'] >= 1986].copy()

print(f"📊 Data loaded: {len(df_model)} years ({df_model['Year'].min():.0f}-{df_model['Year'].max():.0f})")
print(f"\nAvailable indicators:")
for col in numeric_cols:
    valid_count = df_model[col].notna().sum()
    print(f"  • {col}: {valid_count}/{len(df_model)} values ({valid_count/len(df_model)*100:.1f}%)")

# Display recent data
print("\n📈 Recent 5 years:")
print(df_model[['Year', 'GDPTotalBillion', 'GDPGrowthRate', 'InflationRate']].tail().to_string(index=False))

📊 Data loaded: 39 years (1986-2024)

Available indicators:
  • GDPTotalBillion: 39/39 values (100.0%)
  • GDPGrowthRate: 39/39 values (100.0%)
  • InflationRate: 22/39 values (56.4%)
  • GDPPerCapita: 39/39 values (100.0%)
  • UnemploymentRate: 34/39 values (87.2%)
  • FDINetInflowsMillion: 39/39 values (100.0%)
  • ExportsPercentGDP: 38/39 values (97.4%)

📈 Recent 5 years:
 Year  GDPTotalBillion  GDPGrowthRate  InflationRate
 2020          346.616           2.87           3.22
 2021          366.475           2.55           1.83
 2022          413.445           8.54           3.16
 2023          433.858           5.07           3.25
 2024          476.388           7.09           3.62


## 🎯 Phần 1: Dự Báo GDP Total (2025-2030)

### 1.1 Linear & Polynomial Regression

In [3]:
# Prepare GDP data
gdp_data = df_model[['Year', 'GDPTotalBillion']].dropna()

# Split train/test (last 5 years for testing)
train_data = gdp_data[gdp_data['Year'] <= 2019]
test_data = gdp_data[gdp_data['Year'] > 2019]

X_train = train_data[['Year']].values
y_train = train_data['GDPTotalBillion'].values
X_test = test_data[['Year']].values
y_test = test_data['GDPTotalBillion'].values

# Future years to predict
future_years = np.array([[y] for y in range(2025, 2031)])

# Model 1: Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_pred_test = lr_model.predict(X_test)
lr_pred_future = lr_model.predict(future_years)

# Model 2: Polynomial Regression (degree 2)
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
X_future_poly = poly.transform(future_years)

poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
poly_pred_test = poly_model.predict(X_test_poly)
poly_pred_future = poly_model.predict(X_future_poly)

# Model 3: Polynomial degree 3 (for comparison)
poly3 = PolynomialFeatures(degree=3)
X_train_poly3 = poly3.fit_transform(X_train)
X_future_poly3 = poly3.transform(future_years)

poly3_model = LinearRegression()
poly3_model.fit(X_train_poly3, y_train)
poly3_pred_future = poly3_model.predict(X_future_poly3)

# Calculate metrics
print("📊 Model Performance on Test Data (2020-2024):\n")
print(f"Linear Regression:")
print(f"  MAE: ${mean_absolute_error(y_test, lr_pred_test):.2f}B")
print(f"  RMSE: ${np.sqrt(mean_squared_error(y_test, lr_pred_test)):.2f}B")
print(f"  R²: {r2_score(y_test, lr_pred_test):.4f}")

print(f"\nPolynomial (degree 2):")
print(f"  MAE: ${mean_absolute_error(y_test, poly_pred_test):.2f}B")
print(f"  RMSE: ${np.sqrt(mean_squared_error(y_test, poly_pred_test)):.2f}B")
print(f"  R²: {r2_score(y_test, poly_pred_test):.4f}")

# Display predictions
print("\n🔮 GDP Forecasts (2025-2030):\n")
forecast_df = pd.DataFrame({
    'Year': range(2025, 2031),
    'Linear': lr_pred_future,
    'Poly2': poly_pred_future,
    'Poly3': poly3_pred_future
})
print(forecast_df.to_string(index=False, float_format=lambda x: f'${x:.1f}B'))

📊 Model Performance on Test Data (2020-2024):

Linear Regression:
  MAE: $136.34B
  RMSE: $140.52B
  R²: -8.0870

Polynomial (degree 2):
  MAE: $13.14B
  RMSE: $15.07B
  R²: 0.8955

🔮 GDP Forecasts (2025-2030):

 Year  Linear   Poly2   Poly3
 2025 $298.0B $512.3B $546.8B
 2026 $307.0B $545.3B $586.3B
 2027 $316.0B $579.4B $627.6B
 2028 $325.0B $614.5B $670.5B
 2029 $334.0B $650.7B $715.3B
 2030 $342.9B $687.9B $761.7B


### 1.2 ARIMA Model (if available)

In [4]:
if ARIMA_AVAILABLE:
    # Prepare time series data
    gdp_ts = gdp_data.set_index('Year')['GDPTotalBillion']
    
    # Fit ARIMA model (p,d,q) = (1,1,1) - common starting point
    try:
        arima_model = ARIMA(gdp_ts, order=(1, 1, 1))
        arima_fitted = arima_model.fit()
        
        # Forecast
        arima_forecast = arima_fitted.forecast(steps=6)
        arima_pred_future = arima_forecast.values
        
        print("✅ ARIMA(1,1,1) Model Fitted\n")
        print("🔮 ARIMA Forecasts (2025-2030):\n")
        for i, year in enumerate(range(2025, 2031)):
            print(f"  {year}: ${arima_pred_future[i]:.1f}B")
        
        # Add to forecast dataframe
        forecast_df['ARIMA'] = arima_pred_future
        
    except Exception as e:
        print(f"⚠️  ARIMA fitting failed: {e}")
        arima_pred_future = None
else:
    print("⚠️  ARIMA not available - install statsmodels for time series analysis")
    arima_pred_future = None

✅ ARIMA(1,1,1) Model Fitted

🔮 ARIMA Forecasts (2025-2030):

  2025: $506.8B
  2026: $536.7B
  2027: $566.2B
  2028: $595.3B
  2029: $624.0B
  2030: $652.3B


c:\Users\ASUS\miniconda3\envs\py39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\ASUS\miniconda3\envs\py39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\ASUS\miniconda3\envs\py39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
c:\Users\ASUS\miniconda3\envs\py39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported

### 1.3 Facebook Prophet (if available)

In [5]:
if PROPHET_AVAILABLE:
    # Prepare data for Prophet (needs 'ds' and 'y' columns)
    prophet_df = gdp_data.copy()
    prophet_df['ds'] = pd.to_datetime(prophet_df['Year'], format='%Y')
    prophet_df['y'] = prophet_df['GDPTotalBillion']
    prophet_df = prophet_df[['ds', 'y']]
    
    # Fit model
    try:
        prophet_model = Prophet(
            yearly_seasonality=False,
            weekly_seasonality=False,
            daily_seasonality=False,
            changepoint_prior_scale=0.05  # More flexible trend
        )
        prophet_model.fit(prophet_df)
        
        # Create future dataframe
        future = prophet_model.make_future_dataframe(periods=6, freq='Y')
        prophet_forecast = prophet_model.predict(future)
        
        # Extract predictions for 2025-2030
        prophet_pred_future = prophet_forecast[prophet_forecast['ds'].dt.year >= 2025]['yhat'].values[:6]
        
        print("✅ Prophet Model Fitted\n")
        print("🔮 Prophet Forecasts (2025-2030):\n")
        for i, year in enumerate(range(2025, 2031)):
            print(f"  {year}: ${prophet_pred_future[i]:.1f}B")
        
        # Add to forecast dataframe
        forecast_df['Prophet'] = prophet_pred_future
        
    except Exception as e:
        print(f"⚠️  Prophet fitting failed: {e}")
        prophet_pred_future = None
else:
    print("⚠️  Prophet not available - install: pip install prophet")
    prophet_pred_future = None

07:03:25 - cmdstanpy - INFO - Chain [1] start processing
07:03:25 - cmdstanpy - INFO - Chain [1] done processing
07:03:25 - cmdstanpy - INFO - Chain [1] done processing


✅ Prophet Model Fitted

🔮 Prophet Forecasts (2025-2030):

  2025: $497.7B
  2026: $520.8B
  2027: $543.9B
  2028: $567.0B
  2029: $590.1B
⚠️  Prophet fitting failed: index 5 is out of bounds for axis 0 with size 5


### 1.4 Ensemble Forecast & Visualization

In [6]:
# Calculate ensemble (average of all models)
model_cols = [col for col in ['Linear', 'Poly2', 'Poly3', 'ARIMA', 'Prophet'] if col in forecast_df.columns]
forecast_df['Ensemble'] = forecast_df[model_cols].mean(axis=1)
forecast_df['Min'] = forecast_df[model_cols].min(axis=1)
forecast_df['Max'] = forecast_df[model_cols].max(axis=1)

print("\n📊 Final GDP Forecasts with Confidence Range:\n")
for idx, row in forecast_df.iterrows():
    print(f"{row['Year']:.0f}: ${row['Ensemble']:.1f}B (Range: ${row['Min']:.1f}B - ${row['Max']:.1f}B)")

# Create visualization
fig = go.Figure()

# Historical data
fig.add_trace(go.Scatter(
    x=gdp_data['Year'],
    y=gdp_data['GDPTotalBillion'],
    mode='lines+markers',
    name='Lịch sử (1986-2024)',
    line=dict(color='#2c3e50', width=3),
    marker=dict(size=6)
))

# Add each model's forecast
colors = {'Linear': '#e74c3c', 'Poly2': '#3498db', 'Poly3': '#9b59b6', 
          'ARIMA': '#27ae60', 'Prophet': '#f39c12'}

for col in model_cols:
    fig.add_trace(go.Scatter(
        x=forecast_df['Year'],
        y=forecast_df[col],
        mode='lines+markers',
        name=f'{col} (2025-30)',
        line=dict(color=colors.get(col, '#95a5a6'), width=2, dash='dash'),
        marker=dict(size=8)
    ))

# Add ensemble forecast
fig.add_trace(go.Scatter(
    x=forecast_df['Year'],
    y=forecast_df['Ensemble'],
    mode='lines+markers',
    name='Ensemble (TB các mô hình)',
    line=dict(color='#16a085', width=4),
    marker=dict(size=10, symbol='diamond')
))

# Add confidence interval
fig.add_trace(go.Scatter(
    x=forecast_df['Year'].tolist() + forecast_df['Year'].tolist()[::-1],
    y=forecast_df['Max'].tolist() + forecast_df['Min'].tolist()[::-1],
    fill='toself',
    fillcolor='rgba(22, 160, 133, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='Khoảng tin cậy',
    showlegend=True
))

# Add vertical line at 2024
fig.add_vline(
    x=2024.5,
    line_dash="solid",
    line_color="gray",
    line_width=2,
    annotation_text="Dự báo →",
    annotation_position="top"
)

fig.update_layout(
    title={
        'text': '🔮 Dự Báo GDP Việt Nam (2025-2030)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    xaxis_title='Năm',
    yaxis_title='GDP (tỷ USD)',
    height=600,
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255,255,255,0.8)"
    )
)

fig.show()


📊 Final GDP Forecasts with Confidence Range:

2025: $466.0B (Range: $298.0B - $546.8B)
2026: $493.8B (Range: $307.0B - $586.3B)
2027: $522.3B (Range: $316.0B - $627.6B)
2028: $551.3B (Range: $325.0B - $670.5B)
2029: $581.0B (Range: $334.0B - $715.3B)
2030: $611.2B (Range: $342.9B - $761.7B)


## 📈 Phần 2: Dự Báo Tốc Độ Tăng Trưởng GDP

In [7]:
# Prepare growth rate data
growth_data = df_model[['Year', 'GDPGrowthRate']].dropna()

# Recent trend (2010-2024) - more relevant for forecasting
recent_growth = growth_data[growth_data['Year'] >= 2010]

X_growth = recent_growth[['Year']].values
y_growth = recent_growth['GDPGrowthRate'].values

# Linear model for growth rate
growth_model = LinearRegression()
growth_model.fit(X_growth, y_growth)

# Predict future growth rates
growth_pred_future = growth_model.predict(future_years)

# Calculate historical statistics
mean_growth = recent_growth['GDPGrowthRate'].mean()
std_growth = recent_growth['GDPGrowthRate'].std()

# Create scenarios
optimistic = growth_pred_future + std_growth
baseline = growth_pred_future
pessimistic = growth_pred_future - std_growth

print("📊 Historical Growth Statistics (2010-2024):\n")
print(f"  Average: {mean_growth:.2f}%")
print(f"  Std Dev: {std_growth:.2f}%")
print(f"  Min: {recent_growth['GDPGrowthRate'].min():.2f}% (COVID-19)")
print(f"  Max: {recent_growth['GDPGrowthRate'].max():.2f}%")

print("\n🔮 Growth Rate Forecasts (2025-2030):\n")
scenario_df = pd.DataFrame({
    'Year': range(2025, 2031),
    'Optimistic': optimistic,
    'Baseline': baseline,
    'Pessimistic': pessimistic
})
print(scenario_df.to_string(index=False, float_format=lambda x: f'{x:.2f}%'))

# Visualization
fig = go.Figure()

# Historical data
fig.add_trace(go.Scatter(
    x=growth_data['Year'],
    y=growth_data['GDPGrowthRate'],
    mode='lines+markers',
    name='Lịch sử',
    line=dict(color='#2c3e50', width=3),
    marker=dict(size=6)
))

# Scenarios
scenarios = [
    ('Optimistic', optimistic, '#27ae60', 'Lạc quan'),
    ('Baseline', baseline, '#3498db', 'Cơ bản'),
    ('Pessimistic', pessimistic, '#e74c3c', 'Bi quan')
]

for name, values, color, label in scenarios:
    fig.add_trace(go.Scatter(
        x=scenario_df['Year'],
        y=values,
        mode='lines+markers',
        name=f'{label}',
        line=dict(color=color, width=3, dash='dash'),
        marker=dict(size=8)
    ))

# Add target line (6.5% - government target)
fig.add_hline(
    y=6.5,
    line_dash="dot",
    line_color="green",
    annotation_text="Mục tiêu Chính phủ (6.5%)",
    annotation_position="right"
)

fig.add_vline(x=2024.5, line_dash="solid", line_color="gray", line_width=2)

fig.update_layout(
    title={
        'text': '📈 Dự Báo Tốc Độ Tăng Trưởng GDP (2025-2030)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    xaxis_title='Năm',
    yaxis_title='Tăng trưởng GDP (%)',
    height=500,
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

📊 Historical Growth Statistics (2010-2024):

  Average: 6.12%
  Std Dev: 1.63%
  Min: 2.55% (COVID-19)
  Max: 8.54%

🔮 Growth Rate Forecasts (2025-2030):

 Year  Optimistic  Baseline  Pessimistic
 2025       7.49%     5.86%        4.23%
 2026       7.46%     5.83%        4.19%
 2027       7.43%     5.79%        4.16%
 2028       7.39%     5.76%        4.13%
 2029       7.36%     5.73%        4.09%
 2030       7.33%     5.69%        4.06%


## 💰 Phần 3: Dự Báo Lạm Phát & Thất Nghiệp

In [8]:
# Inflation forecast
inflation_data = df_model[['Year', 'InflationRate']].dropna()
recent_inflation = inflation_data[inflation_data['Year'] >= 2010]

X_inf = recent_inflation[['Year']].values
y_inf = recent_inflation['InflationRate'].values

inf_model = LinearRegression()
inf_model.fit(X_inf, y_inf)
inf_pred_future = inf_model.predict(future_years)

# Unemployment forecast
unemp_data = df_model[['Year', 'UnemploymentRate']].dropna()
if len(unemp_data) >= 10:
    recent_unemp = unemp_data[unemp_data['Year'] >= 2010]
    
    X_unemp = recent_unemp[['Year']].values
    y_unemp = recent_unemp['UnemploymentRate'].values
    
    unemp_model = LinearRegression()
    unemp_model.fit(X_unemp, y_unemp)
    unemp_pred_future = unemp_model.predict(future_years)
    
    unemp_available = True
else:
    unemp_available = False
    print("⚠️  Insufficient unemployment data for forecasting")

# Create dual-axis chart
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Lạm Phát (%)', 'Thất Nghiệp (%)' if unemp_available else 'Thất Nghiệp (N/A)'),
    horizontal_spacing=0.12
)

# Inflation
fig.add_trace(
    go.Scatter(
        x=inflation_data['Year'],
        y=inflation_data['InflationRate'],
        mode='lines',
        name='Lịch sử',
        line=dict(color='#2c3e50', width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=future_years.flatten(),
        y=inf_pred_future,
        mode='lines+markers',
        name='Dự báo',
        line=dict(color='#e74c3c', width=3, dash='dash'),
        marker=dict(size=8)
    ),
    row=1, col=1
)

# Unemployment
if unemp_available:
    fig.add_trace(
        go.Scatter(
            x=unemp_data['Year'],
            y=unemp_data['UnemploymentRate'],
            mode='lines',
            name='Lịch sử',
            line=dict(color='#2c3e50', width=2),
            showlegend=False
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=future_years.flatten(),
            y=unemp_pred_future,
            mode='lines+markers',
            name='Dự báo',
            line=dict(color='#f39c12', width=3, dash='dash'),
            marker=dict(size=8),
            showlegend=False
        ),
        row=1, col=2
    )

fig.update_xaxes(title_text="Năm", row=1, col=1)
fig.update_xaxes(title_text="Năm", row=1, col=2)
fig.update_yaxes(title_text="%", row=1, col=1)
fig.update_yaxes(title_text="%", row=1, col=2)

fig.update_layout(
    title={
        'text': '💰 Dự Báo Lạm Phát & Thất Nghiệp',
        'x': 0.5,
        'xanchor': 'center'
    },
    height=400,
    template='plotly_white'
)

fig.show()

print("\n💰 Inflation Forecasts (2025-2030):\n")
for i, year in enumerate(range(2025, 2031)):
    print(f"  {year}: {inf_pred_future[i]:.2f}%")

if unemp_available:
    print("\n👥 Unemployment Forecasts (2025-2030):\n")
    for i, year in enumerate(range(2025, 2031)):
        print(f"  {year}: {unemp_pred_future[i]:.2f}%")


💰 Inflation Forecasts (2025-2030):

  2025: -0.02%
  2026: -0.65%
  2027: -1.29%
  2028: -1.92%
  2029: -2.56%
  2030: -3.19%

👥 Unemployment Forecasts (2025-2030):

  2025: 1.95%
  2026: 2.01%
  2027: 2.06%
  2028: 2.11%
  2029: 2.16%
  2030: 2.21%


## 🌏 Phần 4: Dự Báo GDP Per Capita

In [9]:
# GDP per capita forecast
percap_data = df_model[['Year', 'GDPPerCapita']].dropna()

X_percap = percap_data[['Year']].values
y_percap = percap_data['GDPPerCapita'].values

# Use polynomial for per capita (captures accelerating growth)
poly_percap = PolynomialFeatures(degree=2)
X_percap_poly = poly_percap.fit_transform(X_percap)
X_future_percap = poly_percap.transform(future_years)

percap_model = LinearRegression()
percap_model.fit(X_percap_poly, y_percap)
percap_pred_future = percap_model.predict(X_future_percap)

# Calculate milestones
print("📊 GDP Per Capita Milestones:\n")
current_percap = percap_data.iloc[-1]['GDPPerCapita']
print(f"  2024: ${current_percap:.0f}")

for i, year in enumerate(range(2025, 2031)):
    value = percap_pred_future[i]
    milestone = ""
    if value >= 5000 and current_percap < 5000:
        milestone = " ⭐ Reach $5,000"
    elif value >= 6000 and percap_pred_future[i-1] < 6000 if i > 0 else False:
        milestone = " ⭐ Reach $6,000"
    elif value >= 7000 and percap_pred_future[i-1] < 7000 if i > 0 else False:
        milestone = " ⭐ Reach $7,000"
    
    print(f"  {year}: ${value:.0f}{milestone}")

# Visualization
fig = go.Figure()

# Historical
fig.add_trace(go.Bar(
    x=percap_data['Year'],
    y=percap_data['GDPPerCapita'],
    name='Lịch sử',
    marker_color='#3498db',
    opacity=0.7
))

# Forecast
fig.add_trace(go.Bar(
    x=future_years.flatten(),
    y=percap_pred_future,
    name='Dự báo',
    marker_color='#27ae60',
    opacity=0.7
))

# Add milestone lines
milestones = [5000, 6000, 7000]
for milestone in milestones:
    fig.add_hline(
        y=milestone,
        line_dash="dot",
        line_color="red",
        annotation_text=f"${milestone:,}",
        annotation_position="right"
    )

fig.update_layout(
    title={
        'text': '🌏 Dự Báo GDP Bình Quân Đầu Người (2025-2030)',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Năm',
    yaxis_title='GDP Per Capita (USD)',
    height=500,
    template='plotly_white',
    barmode='group'
)

fig.show()

📊 GDP Per Capita Milestones:

  2024: $4717
  2025: $5051 ⭐ Reach $5,000
  2026: $5361 ⭐ Reach $5,000
  2027: $5680 ⭐ Reach $5,000
  2028: $6009 ⭐ Reach $5,000
  2029: $6347 ⭐ Reach $5,000
  2030: $6695 ⭐ Reach $5,000


## 📊 Phần 5: Kịch Bản Tổng Hợp (2025-2030)

In [10]:
# Create comprehensive scenario analysis
scenarios_full = pd.DataFrame({
    'Year': range(2025, 2031)
})

# Baseline scenario (ensemble forecast)
scenarios_full['GDP_Baseline'] = forecast_df['Ensemble'].values
scenarios_full['Growth_Baseline'] = scenario_df['Baseline'].values
scenarios_full['Inflation_Baseline'] = inf_pred_future
scenarios_full['PerCapita_Baseline'] = percap_pred_future

# Optimistic scenario (+1 std dev on growth)
# Calculate optimistic GDP based on higher growth rates
gdp_2024 = gdp_data.iloc[-1]['GDPTotalBillion']
gdp_optimistic = [gdp_2024]
for growth in scenario_df['Optimistic'].values:
    gdp_optimistic.append(gdp_optimistic[-1] * (1 + growth/100))
scenarios_full['GDP_Optimistic'] = gdp_optimistic[1:]
scenarios_full['Growth_Optimistic'] = scenario_df['Optimistic'].values

# Pessimistic scenario (-1 std dev on growth)
gdp_pessimistic = [gdp_2024]
for growth in scenario_df['Pessimistic'].values:
    gdp_pessimistic.append(gdp_pessimistic[-1] * (1 + growth/100))
scenarios_full['GDP_Pessimistic'] = gdp_pessimistic[1:]
scenarios_full['Growth_Pessimistic'] = scenario_df['Pessimistic'].values

print("="*80)
print("  🎯 KỊCH BẢN DỰ BÁO KINH TẾ VIỆT NAM (2025-2030)")
print("="*80)

print("\n📊 KỊCH BẢN CƠ BẢN (Baseline):")
print("\nNăm | GDP (tỷ$) | Tăng trưởng (%) | Lạm phát (%) | GDP/người ($)")
print("-" * 70)
for idx, row in scenarios_full.iterrows():
    print(f"{row['Year']:.0f}  | ${row['GDP_Baseline']:>7.1f}    | "
          f"{row['Growth_Baseline']:>6.2f}          | "
          f"{row['Inflation_Baseline']:>5.2f}        | "
          f"${row['PerCapita_Baseline']:>6.0f}")

print("\n📈 KỊCH BẢN LẠC QUAN (Optimistic):")
print("\nNăm | GDP (tỷ$) | Tăng trưởng (%)")
print("-" * 40)
for idx, row in scenarios_full.iterrows():
    print(f"{row['Year']:.0f}  | ${row['GDP_Optimistic']:>7.1f}    | {row['Growth_Optimistic']:>6.2f}")

print("\n📉 KỊCH BẢN BI QUAN (Pessimistic):")
print("\nNăm | GDP (tỷ$) | Tăng trưởng (%)")
print("-" * 40)
for idx, row in scenarios_full.iterrows():
    print(f"{row['Year']:.0f}  | ${row['GDP_Pessimistic']:>7.1f}    | {row['Growth_Pessimistic']:>6.2f}")

print("\n" + "="*80)
print("\n🎯 CÁC MỐC QUAN TRỌNG:\n")
gdp_2030_baseline = scenarios_full.iloc[-1]['GDP_Baseline']
gdp_2030_optimistic = scenarios_full.iloc[-1]['GDP_Optimistic']
gdp_2030_pessimistic = scenarios_full.iloc[-1]['GDP_Pessimistic']

print(f"  GDP 2030 (Baseline): ${gdp_2030_baseline:.1f}B")
print(f"  GDP 2030 (Optimistic): ${gdp_2030_optimistic:.1f}B")
print(f"  GDP 2030 (Pessimistic): ${gdp_2030_pessimistic:.1f}B")
print(f"\n  Tăng trưởng TB 2025-2030: {scenarios_full['Growth_Baseline'].mean():.2f}%")
print(f"  Lạm phát TB 2025-2030: {scenarios_full['Inflation_Baseline'].mean():.2f}%")
print(f"  GDP/người 2030: ${scenarios_full.iloc[-1]['PerCapita_Baseline']:.0f}")

# Calculate CAGR
years = 6
cagr_baseline = ((gdp_2030_baseline / gdp_2024) ** (1/years) - 1) * 100
cagr_optimistic = ((gdp_2030_optimistic / gdp_2024) ** (1/years) - 1) * 100
cagr_pessimistic = ((gdp_2030_pessimistic / gdp_2024) ** (1/years) - 1) * 100

print(f"\n  CAGR 2025-2030:")
print(f"    Baseline: {cagr_baseline:.2f}%")
print(f"    Optimistic: {cagr_optimistic:.2f}%")
print(f"    Pessimistic: {cagr_pessimistic:.2f}%")
print("\n" + "="*80)

  🎯 KỊCH BẢN DỰ BÁO KINH TẾ VIỆT NAM (2025-2030)

📊 KỊCH BẢN CƠ BẢN (Baseline):

Năm | GDP (tỷ$) | Tăng trưởng (%) | Lạm phát (%) | GDP/người ($)
----------------------------------------------------------------------
2025  | $  466.0    |   5.86          | -0.02        | $  5051
2026  | $  493.8    |   5.83          | -0.65        | $  5361
2027  | $  522.3    |   5.79          | -1.29        | $  5680
2028  | $  551.3    |   5.76          | -1.92        | $  6009
2029  | $  581.0    |   5.73          | -2.56        | $  6347
2030  | $  611.2    |   5.69          | -3.19        | $  6695

📈 KỊCH BẢN LẠC QUAN (Optimistic):

Năm | GDP (tỷ$) | Tăng trưởng (%)
----------------------------------------
2025  | $  512.1    |   7.49
2026  | $  550.3    |   7.46
2027  | $  591.1    |   7.43
2028  | $  634.8    |   7.39
2029  | $  681.5    |   7.36
2030  | $  731.5    |   7.33

📉 KỊCH BẢN BI QUAN (Pessimistic):

Năm | GDP (tỷ$) | Tăng trưởng (%)
----------------------------------------
2025  | $

## 🎨 Phần 6: Dashboard Tổng Quan

In [11]:
# Create comprehensive dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'GDP Total (tỷ USD)',
        'Tăng Trưởng (%)',
        'GDP/Người (USD)',
        'Lạm Phát (%)'
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.12
)

# 1. GDP Total - 3 scenarios
for scenario, color, name in [('GDP_Baseline', '#3498db', 'Cơ bản'),
                               ('GDP_Optimistic', '#27ae60', 'Lạc quan'),
                               ('GDP_Pessimistic', '#e74c3c', 'Bi quan')]:
    fig.add_trace(
        go.Scatter(
            x=scenarios_full['Year'],
            y=scenarios_full[scenario],
            mode='lines+markers',
            name=name,
            line=dict(color=color, width=2),
            showlegend=True
        ),
        row=1, col=1
    )

# 2. Growth Rate - 3 scenarios
for scenario, color in [('Growth_Baseline', '#3498db'),
                        ('Growth_Optimistic', '#27ae60'),
                        ('Growth_Pessimistic', '#e74c3c')]:
    fig.add_trace(
        go.Scatter(
            x=scenarios_full['Year'],
            y=scenarios_full[scenario],
            mode='lines+markers',
            line=dict(color=color, width=2),
            showlegend=False
        ),
        row=1, col=2
    )

# 3. Per Capita
fig.add_trace(
    go.Bar(
        x=scenarios_full['Year'],
        y=scenarios_full['PerCapita_Baseline'],
        marker_color='#9b59b6',
        showlegend=False
    ),
    row=2, col=1
)

# 4. Inflation
fig.add_trace(
    go.Scatter(
        x=scenarios_full['Year'],
        y=scenarios_full['Inflation_Baseline'],
        mode='lines+markers',
        line=dict(color='#f39c12', width=3),
        fill='tozeroy',
        fillcolor='rgba(243, 156, 18, 0.2)',
        showlegend=False
    ),
    row=2, col=2
)

# Update axes
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(title_text="Năm", row=i, col=j)

fig.update_layout(
    title={
        'text': '🎨 Dashboard Dự Báo Kinh Tế Việt Nam (2025-2030)',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 22}
    },
    height=700,
    template='plotly_white',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

## 📝 Phần 7: Kết Luận & Khuyến Nghị

In [12]:
print("="*80)
print("  📝 KẾT LUẬN & KHUYẾN NGHỊ")
print("="*80)

print("\n🎯 DỰ BÁO CHÍNH:\n")
print(f"  1. GDP 2030 dự kiến đạt ${gdp_2030_baseline:.1f}B (baseline)")
print(f"     → Tăng {((gdp_2030_baseline/gdp_2024 - 1) * 100):.1f}% so với 2024")
print(f"\n  2. Tăng trưởng TB 2025-2030: {scenarios_full['Growth_Baseline'].mean():.2f}%/năm")
print(f"     → Cao hơn mục tiêu Chính phủ (6.5%)" if scenarios_full['Growth_Baseline'].mean() > 6.5 else "     → Cần nỗ lực để đạt mục tiêu 6.5%")
print(f"\n  3. GDP/người 2030: ${scenarios_full.iloc[-1]['PerCapita_Baseline']:.0f}")
print(f"     → Tăng {((scenarios_full.iloc[-1]['PerCapita_Baseline']/current_percap - 1) * 100):.1f}% so với 2024")
print(f"\n  4. Lạm phát dự kiến: {scenarios_full['Inflation_Baseline'].mean():.2f}% (TB)")
print(f"     → Ổn định trong khoảng kiểm soát của NHNN")

print("\n⚠️  RỦI RO CẦN LƯU Ý:\n")
print("  • Bất ổn địa chính trị toàn cầu")
print("  • Thay đổi chính sách thương mại quốc tế")
print("  • Biến động giá nguyên liệu, năng lượng")
print("  • Tác động biến đổi khí hậu")
print("  • Cạnh tranh khu vực gia tăng")

print("\n✅ KHUYẾN NGHỊ:\n")
print("  1. Đa dạng hóa thị trường xuất khẩu")
print("  2. Đầu tư mạnh vào R&D, đổi mới sáng tạo")
print("  3. Nâng cao chất lượng nguồn nhân lực")
print("  4. Phát triển công nghiệp công nghệ cao")
print("  5. Cải thiện môi trường kinh doanh")
print("  6. Thúc đẩy chuyển đổi số, xanh hóa kinh tế")

print("\n🎓 LƯU Ý VỀ DỰ BÁO:\n")
print("  • Dự báo dựa trên xu hướng lịch sử (1986-2024)")
print("  • Giả định điều kiện kinh tế ổn định")
print("  • Không dự đoán được sự kiện bất thường (Black Swan)")
print("  • Nên cập nhật mô hình định kỳ với dữ liệu mới")
print("  • Sử dụng kết hợp nhiều mô hình để tăng độ tin cậy")

print("\n" + "="*80)
print("  ✨ Việt Nam có tiềm năng tăng trưởng mạnh mẽ đến 2030")
print("  🚀 Cần chính sách phù hợp để tối ưu hóa cơ hội phát triển")
print("="*80)

  📝 KẾT LUẬN & KHUYẾN NGHỊ

🎯 DỰ BÁO CHÍNH:

  1. GDP 2030 dự kiến đạt $611.2B (baseline)
     → Tăng 28.3% so với 2024

  2. Tăng trưởng TB 2025-2030: 5.78%/năm
     → Cần nỗ lực để đạt mục tiêu 6.5%

  3. GDP/người 2030: $6695
     → Tăng 41.9% so với 2024

  4. Lạm phát dự kiến: -1.61% (TB)
     → Ổn định trong khoảng kiểm soát của NHNN

⚠️  RỦI RO CẦN LƯU Ý:

  • Bất ổn địa chính trị toàn cầu
  • Thay đổi chính sách thương mại quốc tế
  • Biến động giá nguyên liệu, năng lượng
  • Tác động biến đổi khí hậu
  • Cạnh tranh khu vực gia tăng

✅ KHUYẾN NGHỊ:

  1. Đa dạng hóa thị trường xuất khẩu
  2. Đầu tư mạnh vào R&D, đổi mới sáng tạo
  3. Nâng cao chất lượng nguồn nhân lực
  4. Phát triển công nghiệp công nghệ cao
  5. Cải thiện môi trường kinh doanh
  6. Thúc đẩy chuyển đổi số, xanh hóa kinh tế

🎓 LƯU Ý VỀ DỰ BÁO:

  • Dự báo dựa trên xu hướng lịch sử (1986-2024)
  • Giả định điều kiện kinh tế ổn định
  • Không dự đoán được sự kiện bất thường (Black Swan)
  • Nên cập nhật mô hình đ

## 📚 Phụ Lục: Model Comparison

In [13]:
# Compare model predictions visually
print("📊 SO SÁNH CÁC MÔ HÌNH DỰ BÁO GDP:\n")

comparison = forecast_df[['Year'] + model_cols + ['Ensemble']].copy()
print(comparison.to_string(index=False, float_format=lambda x: f'${x:.1f}B'))

print("\n📈 Độ chênh lệch giữa các mô hình:\n")
for year in range(2025, 2031):
    row = comparison[comparison['Year'] == year]
    values = row[model_cols].values[0]
    spread = values.max() - values.min()
    spread_pct = (spread / values.mean()) * 100
    print(f"{year}: ${spread:.1f}B ({spread_pct:.1f}%)")

print("\n✅ Mô hình Ensemble (trung bình) thường cho kết quả ổn định nhất!")

📊 SO SÁNH CÁC MÔ HÌNH DỰ BÁO GDP:

 Year  Linear   Poly2   Poly3   ARIMA  Ensemble
 2025 $298.0B $512.3B $546.8B $506.8B   $466.0B
 2026 $307.0B $545.3B $586.3B $536.7B   $493.8B
 2027 $316.0B $579.4B $627.6B $566.2B   $522.3B
 2028 $325.0B $614.5B $670.5B $595.3B   $551.3B
 2029 $334.0B $650.7B $715.3B $624.0B   $581.0B
 2030 $342.9B $687.9B $761.7B $652.3B   $611.2B

📈 Độ chênh lệch giữa các mô hình:

2025: $248.8B (53.4%)
2026: $279.4B (56.6%)
2027: $311.6B (59.7%)
2028: $345.6B (62.7%)
2029: $381.3B (65.6%)
2030: $418.8B (68.5%)

✅ Mô hình Ensemble (trung bình) thường cho kết quả ổn định nhất!
